In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(".env"))  # read local .env file

In [ ]:
from pathlib import Path

In [ ]:
from langchain_openai import ChatOpenAI
from pdf_parser.pdf_processor import MathpixPdfConverter

In [ ]:
vision_model = ChatOpenAI(model_name="gpt-4-vision-preview")
text_model = ChatOpenAI(model_name="gpt-4-1106", temperature=0)

In [ ]:
converter = MathpixPdfConverter(text_model=text_model, vision_model=vision_model)

In [ ]:
inbox_path = Path("./inbox")
fulltext_path = Path("./fulltext")
fulltext_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for pdf_path in inbox_path.glob("*.pdf"):
    pdf_result = converter.convert(pdf_path)
    with fulltext_path.joinpath(f"{pdf_path.stem}.txt").open("w") as f:
        f.write(pdf_result.content)

In [ ]:
from rag_factory import build_query_engine

In [ ]:
query_engine = build_query_engine(fulltext_path.as_posix())

In [ ]:
response = query_engine.query("How to action unit activations correspond to stress?")
print(response)